In [ ]:
!pip install torcheval

In [ ]:
import torch
from torcheval.metrics.functional import multiclass_f1_score
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as T
from PIL import Image

In [ ]:
!pip install gdown

In [ ]:
#подгрузка с личного google диска
!gdown 1t5ieuHo8sDAG4W-3udcmb3vsdB0LFNsh

In [ ]:
!unzip TrueDataset.zip

In [ ]:
# подгрузка разметки под класс 0
unnamed_labels = pd.read_csv('/kaggle/working/TrueDataset/train/_annotations2.csv', index_col='Unnamed: 0')
unnamed_labels.groupby('class').count()

**В папке TrueDataset/train/ находится 2 файла аннотаций. Первый это разметка на 1, 2, 3 классы, второй разметка под нулевой класс.**

In [ ]:
# подгрузка разметки под другие классы
labels = pd.read_csv('/kaggle/working/TrueDataset/train/_annotations.csv')
labels.groupby('class').count()

In [ ]:
labels = pd.concat([unnamed_labels, labels])
labels.groupby('class').count()

In [ ]:
#ранжирование
replace_dict = {'Moderaterotation': 2, 'Severerotation': 3, 'Minorrotation': 1, 'Undamage': 0}
labels['class'] = labels['class'].replace(replace_dict)
labels.groupby('class').count()

In [ ]:
# заранее разделение на validation и train
val_labels = labels.sample(frac = 0.2)
train_labels = labels.drop(val_labels.index)

In [ ]:
class Dataset(torch.utils.data.Dataset):
  def __init__(self, data, path='/', is_val=False):
    self.x_paths = [data.iloc[i, 0] for i in range(len(data))]
    self.labels = [data.iloc[i, 3] for i in range(len(data))]
    self.path = path
    self.is_val = is_val
  
  def __len__(self):
    return len(self.x_paths)
  
  def __getitem__(self, idx):
    transform = T.RandomApply([
        T.RandomHorizontalFlip(),
        T.RandomGrayscale(p=0.9),
        T.RandomPerspective(distortion_scale=0.4, p=0.9),
    ], p=0.5)
    x = T.functional.pil_to_tensor(Image.open(self.path + self.x_paths[idx]))
    x = x / 255
    y = self.labels[idx]
    if not self.is_val:
        x = transform(x)
    return x, y

In [ ]:
# параметр is_val=True отменяет transforms для val_dataset
train_dataset = Dataset(train_labels, '/kaggle/working/TrueDataset/train/')
val_dataset = Dataset(val_labels, '/kaggle/working/TrueDataset/train/', is_val=True)

In [ ]:
batch_size = 64
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
def show_images(images, labels):
    f, axes= plt.subplots(1, 5, figsize=(30,5))

    for i, axis in enumerate(axes):
        img = images[i].numpy()
        img = np.transpose(img, (1, 2, 0))

        axes[i].imshow(img)
        axes[i].set_title(labels[i])

    plt.show()


for batch in train_loader:
    images, labels = batch
    break
show_images(images, labels)


In [ ]:
from tqdm.autonotebook import tqdm
from IPython.display import clear_output
import torch.nn as nn
import torchvision.models as models

In [ ]:
#папка сохранение моделей
!mkdir models

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
model = models.efficientnet.efficientnet_b0(pretrained=True)

for p in model.parameters():
    p.requires_grad = False 

model.classifier = torch.nn.Sequential(
    torch.nn.Dropout(p=0.4, inplace=True),
    torch.nn.ReLU(),
    torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=4, bias=True),
)

for p in model.classifier.parameters():
    p.requires_grad = True

model.to(device)
None

In [ ]:
def draw_plots(train_losses, val_losses, train_acc, val_acc):
    plt.figure(figsize=(15, 3))
    plt.subplot(1, 4, 1)
    plt.title('Train loss')
    plt.plot(train_losses)
    plt.grid()

    plt.subplot(1, 4, 2)
    plt.title('Val loss')
    plt.plot(val_losses)
    plt.grid()

    plt.subplot(1, 4, 3)
    plt.title('Train accuracy')
    plt.plot(train_acc)
    plt.grid()

    plt.subplot(1, 4, 4)
    plt.title('Val accuracy')
    plt.plot(val_acc)
    plt.grid()
    plt.show()

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()

learning_rate = 3e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)

In [ ]:
def train(model, loss_fn, scheduler, optimizer, monitoring=True, n_epoch=3, device='cuda'):
    train_losses = []
    val_losses = []
    train_acc = []
    val_acc = []
    val_f1 = []

    max_f1 = 0

    for epoch in range(n_epoch):
        print("Epoch:", epoch)

        # разморозка по 1 слою каждую эпоху вплоть до 4 слоев
        for layer in model.features[max(9 - epoch, 4):]:
            for param in layer.parameters():
                param.requires_grad = True

        if monitoring:
            draw_plots(train_losses, val_losses, train_acc, val_acc)
        

        model.train(True)
        for batch in tqdm(train_loader):
            X_batch, y_batch = batch
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            logits = model(X_batch)

            loss = loss_fn(logits, y_batch)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            train_losses.append(loss.item())

            model_answers = torch.argmax(logits, dim=1)
            train_accuracy = torch.sum(y_batch == model_answers) / len(y_batch)
            train_acc.append(train_accuracy.item())


        model.eval()

        val_accuracy, val_loss, val_f1_score = evaluate(model, val_loader, loss_fn=loss_fn, device=device)
        clear_output(wait=True)
        print('F1_score: ', val_f1_score)

        if max_f1 < val_f1_score:
            print('Save model')
            max_f1 = val_f1_score
            torch.save(model.state_dict(), f'/kaggle/working/models/model_f1:{val_f1_score:.2f}.pth')


        val_losses.append(val_loss.item())
        val_acc.append(val_accuracy)

        scheduler.step(val_loss)

def evaluate(model, dataloader, loss_fn, device):
    losses = []
    num_correct = 0
    num_elements = 0
    f1_score = 0

    for batch in tqdm(dataloader):
        X_batch, y_batch = batch
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        with torch.no_grad():
            logits = model(X_batch)

            loss = loss_fn(logits, y_batch)
            losses.append(loss.item())

            y_pred = torch.argmax(logits, dim=1)

            f1_score += multiclass_f1_score(y_pred, y_batch, num_classes=4)
            
            num_elements += len(y_batch)
            num_correct += torch.sum(y_pred == y_batch)

    accuracy = num_correct / num_elements
    f1_score = f1_score / len(dataloader)

    return accuracy.item(), np.mean(losses), f1_score.item()

In [ ]:
train(model, loss_fn, scheduler, optimizer, n_epoch=10, device=device)

In [ ]:
train_accuracy, train_loss, train_f1 = evaluate(model, train_loader, loss_fn, device=device)
print('Train accuracy is', train_accuracy)
print('Train loss is', train_loss)
print('Train f1 is', train_f1)

In [ ]:
val_accuracy, val_loss, val_f1 = evaluate(model, val_loader, loss_fn, device=device)
print('Val accuracy is', val_accuracy)
print('Val loss is', val_loss)
print('Val f1 is', val_f1)

In [ ]:
# функция предсказывающая несколько картинок из датасета
import random
def predict_images(model, dataset, device):
    plt.figure(figsize=(15, 3))
    with torch.no_grad():
        for idx in range(5):
            img, label_idx = dataset[random.randint(0, len(dataset)-1)]
            img = img.float()
            img = torch.unsqueeze(img, dim=0).to(device)
            label = label_idx

            pred = torch.argmax(model(img)).item()
            pred_label = pred

            plt.subplot(1, 5, idx+1)
            plt.title(f'{label} vs {pred_label}')
            img = img.cpu().detach().numpy()[0]
            img = np.moveaxis(img, 0, 2)
            plt.imshow(img)
            plt.xticks([])
            plt.yticks([])

In [ ]:
predict_images(model, val_dataset, device)

In [ ]:
# функция для предсказания на любой пользовательской картинке
def pred_one_image(path, model, device):
    transform = T.Compose([
        T.ToTensor(),
        T.Resize(416)
    ])
    img = Image.open(path)
    img = transform(img).to(device)
    img = torch.unsqueeze(img, 0)
    pred = model(torch.Tensor(img))
    return torch.argmax(pred, dim=1)

In [ ]:
pred_one_image('/kaggle/input/real-tests/images', model, device)